# Data extraction

In [1]:
import fitz

def pdf_extract(pdf_path):
    pdf_document = fitz.open(pdf_path)
    text = ""
    for page_num in range(pdf_document.page_count):
        page = pdf_document.load_page(page_num)
        text += page.get_text()
    
    return text

In [2]:
#Read all the PDF files
aus = pdf_extract('australia/australia.pdf')
bela = pdf_extract('belarus/belarus_9.pdf')
china = pdf_extract('china/China_7.pdf') + " " + pdf_extract('china/china_8.pdf') + " " + pdf_extract('china/china_9.pdf')
eth = pdf_extract('ethiopia/eth_7.pdf') + " " + pdf_extract('ethiopia/eth_8.pdf') + " " + pdf_extract('ethiopia/eth_9.pdf')
ger = pdf_extract('germany/germany_7.pdf') + " " + pdf_extract('germany/germany_81.pdf') + " " + pdf_extract('germany/germany_82.pdf')+ " "+ pdf_extract('germany/germany_83.pdf')
india = pdf_extract('india/India_7.pdf') +" "+ pdf_extract('india/India_8.pdf')+" "+ pdf_extract('india/India_9.pdf')
iran = pdf_extract('iran/iran_7.pdf')+" "+ pdf_extract('iran/iran_8.pdf')+" "+pdf_extract('iran/iran_9.pdf')
japn = pdf_extract('japan/japan_7.pdf')+" "+pdf_extract('japan/japan_8.pdf')+" "+pdf_extract('japan/japan_9.pdf')
malay = pdf_extract('malay/malay_7.pdf')
with open('mexico/Mexico_7.txt','r', encoding = 'utf-8', errors = 'ignore') as file:
    mexico_7 = file.read()
with open('mexico/Mexico_8.txt','r', encoding = 'utf-8', errors = 'ignore') as file:
    mexico_8 = file.read()
mex = mexico_7 +" "+ mexico_8
phili = pdf_extract('phillipeans/Phili_7.pdf') +" "+pdf_extract('phillipeans/phili_8.pdf')+" "+pdf_extract('phillipeans/Phili_9.pdf')
saudi = pdf_extract('saudi/Saudi_7.pdf')+" "+ pdf_extract('saudi/Saudi_8.pdf')+" "+ pdf_extract('saudi/Saudi_9.pdf')
sc = pdf_extract('southernCone/sc_co_7.pdf')+" "+pdf_extract('southernCone/sc_ft_8.pdf')+" "+pdf_extract('southernCone/sc_ft_9.pdf')+" "+pdf_extract('southernCone/sc_nh_8.pdf')+" "+pdf_extract('southernCone/sc_nh_9.pdf')+" "+pdf_extract('southernCone/sc_no_7.pdf')+" "+pdf_extract('southernCone/sc_no_8.pdf')+" "+pdf_extract('southernCone/sc_no_9.pdf')+" "+pdf_extract('southernCone/sc_ps_7.pdf')+" "+pdf_extract('southernCone/sc_ss_7.pdf')
spain = pdf_extract('spain/spain_7.pdf')+" "+pdf_extract('spain/spain_71.pdf')+" "+pdf_extract('spain/spain_72.pdf')+" "+pdf_extract('spain/spain_8.pdf')+" "+pdf_extract('spain/spain_9.pdf')
sa = pdf_extract('southAfri/sa_7.pdf')+" "+pdf_extract('southAfri/sa_9.pdf')
ssu = pdf_extract('Ssudan/ssudan_7.pdf')+" "+pdf_extract('Ssudan/ssudan_8.pdf')+" "+pdf_extract('Ssudan/ssudan_9.pdf')
thai = pdf_extract('thailand/thai_7.pdf')
tun = pdf_extract('Tunisia/Tun_7.pdf')+" "+pdf_extract('Tunisia/Tun_8.pdf')+" "+pdf_extract('Tunisia/Tun_9.pdf')
turk = pdf_extract('turkey/Turkey_9.pdf')
uk = pdf_extract('UK/UK_7.pdf')+" "+pdf_extract('UK/UK_8.pdf')+" "+pdf_extract('UK/UK_9.pdf')
ukr = pdf_extract('Ukraine/ukraine_7.pdf')+" "+pdf_extract('Ukraine/ukraine_8.pdf')+" "+pdf_extract('Ukraine/ukraine_9.pdf')
uzi = pdf_extract('Uzbekistan/Uzi_7.pdf')+" "+pdf_extract('Uzbekistan/Uzi_9.pdf')
with open('vietnam/Viet_7.txt','r')as file:
    viet_7 = file.read()
with open('vietnam/Viet_8.txt', 'r', encoding='utf-8', errors='ignore') as file:
    viet_8 = file.read()
viet = viet_7 +" "+ viet_8

MuPDF error: syntax error: unknown cid font type

MuPDF error: syntax error: unknown cid font type

MuPDF error: syntax error: could not parse color space (240438 0 R)



In [3]:
#Put them into a list
textbooks = [aus, bela, china, eth, ger, india, iran, japn, malay, mex, phili, saudi, sc, spain, sa, ssu, thai, tun, turk, uk, ukr, uzi, viet]

In [4]:
#Categorize them according to their cultural sphere
sino = [china, viet, japn]
indo = [india, thai, malay]
islam = [saudi, iran, tun, uzi]
east = [turk, bela, ukr]
afri = [ssu, eth, sa]
west = [ger, spain, phili,uk, aus]
latin = [mex, sc]
sphere = [sino, indo, islam,east, afri, west, latin]

# Data Cleaning

In [5]:
import re
from langdetect import detect
from langdetect import LangDetectException
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [6]:
def clean_text(post):
    if detect(post) != 'en':
        return ''
    # Make posts lowercase
    post = post.lower()

    # Remove punctuation
    post = re.sub(r'[^a-zA-Z\s]', ' ', post)
    
    # Remove words with repeated letters
    #post = re.sub(r'([a-zA-Z])\1+', r'\1', post)
    
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    word_tokens = word_tokenize(post)
    filtered_text = [word for word in word_tokens if word not in stop_words]
    
    return ' '.join(filtered_text)

In [7]:
cleanTextbooks = []
for i in textbooks:
    cleanTextbooks.append(clean_text(i))
cleanSphere = []
for i in sphere:
    for country in i:
        cleanSphere.append(clean_text(country))

In [8]:
#These have spaces, we use these to select the male and female words
menWords = [" man ", " boy ", " male ", " brother ", " father ", " son ", " husband ", " king ", " prince ", " uncle ", " nephew ", " he ", " him ", " his ", " gentleman ", " sir ", " mr. ", " hero ", " lord ", " patriarch ", " men "]
womenWords = [" woman ", " girl ", " female ", " sister ", " mother ", " daughter ", " wife ", " queen ", " princess ", " aunt ", " niece ", " she ", " her ", " hers ", " lady ", " ma'am "," madam ", " mrs. ", " ms. ", " miss ", " heroine ", " dame ", " matriarch ", " women "]
#These have no spaces, so these can be used to throw away words during TFIDF
menWordsNoSpace = ["man", "boy", "male", "brother", "father", "son", "husband", "king", "prince", "uncle", "nephew", "he", "him", "his", "gentleman", "sir", "mr", "hero", "lord", "patriarch", "men"]
womenWordsNoSpace = ["woman", "girl", "female", "sister", "mother", "daughter", "wife", "queen", "princess", "aunt", "niece", "she", "her", "hers", "lady", "ma'am","madam", "mrs", "heroine", "dame", "matriarch", "women"]

In [9]:
menPara = [[] for _ in range(23)]
womenPara = [[] for _ in range(23)]
for i in range(0, 23):
    text = cleanTextbooks[i]
    paragraphs = [text[j:j+100] for j in range(0, len(text), 100)]
    for paragraph in paragraphs:
        for j in menWords:
            if j in paragraph:
                menPara[i].append(paragraph)
        for j in womenWords:
            if j in paragraph:
                womenPara[i].append(paragraph)

# Occurence

In [10]:
men_count = sum(len(sublist) for sublist in menPara)
print(men_count)
women_count = sum(len(sublist) for sublist in womenPara)
print(women_count)

4730
3718


# Firstness

In [11]:
femaleFirst = []
for sublist in womenPara:  
    for paragraph in sublist:  
        if isinstance(paragraph, str):  
            words = paragraph.split()  
            for i in range(len(words) - 1): 
                if words[i].lower() in womenWordsNoSpace and words[i + 1].lower() in menWordsNoSpace:
                    femaleFirst.append((words[i], words[i + 1]))
print(len(femaleFirst),femaleFirst)

234 [('wife', 'mr'), ('wife', 'mr'), ('wife', 'mr'), ('female', 'male'), ('girl', 'father'), ('mother', 'boy'), ('mother', 'son'), ('sister', 'uncle'), ('women', 'men'), ('girl', 'father'), ('girl', 'father'), ('mother', 'father'), ('mother', 'father'), ('wife', 'husband'), ('woman', 'husband'), ('women', 'men'), ('daughter', 'son'), ('woman', 'man'), ('mother', 'father'), ('mother', 'brother'), ('women', 'men'), ('niece', 'nephew'), ('niece', 'nephew'), ('woman', 'men'), ('girl', 'man'), ('woman', 'man'), ('woman', 'boy'), ('girl', 'man'), ('mother', 'mr'), ('wife', 'brother'), ('sister', 'son'), ('mother', 'son'), ('mother', 'son'), ('mother', 'son'), ('mother', 'son'), ('mother', 'son'), ('mother', 'son'), ('sister', 'brother'), ('daughter', 'father'), ('daughter', 'father'), ('princess', 'king'), ('princess', 'king'), ('mother', 'father'), ('girl', 'mr'), ('wife', 'boy'), ('mother', 'brother'), ('mother', 'brother'), ('sister', 'father'), ('sister', 'father'), ('wife', 'prince'), (

In [12]:
maleFirst = []
for sublist in menPara:  
    for paragraph in sublist:  
        if isinstance(paragraph, str):  
            words = paragraph.split()  
            for i in range(len(words) - 1):  
                if words[i].lower() in menWordsNoSpace and words[i + 1].lower() in womenWordsNoSpace:
                    maleFirst.append((words[i], words[i + 1]))
print(len(maleFirst),maleFirst)

358 [('male', 'female'), ('male', 'female'), ('male', 'female'), ('boy', 'girl'), ('men', 'women'), ('male', 'female'), ('male', 'female'), ('son', 'daughter'), ('son', 'daughter'), ('son', 'daughter'), ('son', 'daughter'), ('male', 'female'), ('male', 'female'), ('king', 'queen'), ('man', 'mother'), ('man', 'mother'), ('prince', 'princess'), ('uncle', 'aunt'), ('uncle', 'aunt'), ('boy', 'girl'), ('brother', 'sister'), ('brother', 'sister'), ('men', 'women'), ('husband', 'wife'), ('brother', 'sister'), ('man', 'wife'), ('man', 'wife'), ('men', 'women'), ('men', 'women'), ('man', 'wife'), ('uncle', 'aunt'), ('uncle', 'aunt'), ('son', 'daughter'), ('son', 'daughter'), ('men', 'women'), ('father', 'mother'), ('father', 'mother'), ('father', 'mother'), ('brother', 'sister'), ('boy', 'girl'), ('father', 'mother'), ('male', 'female'), ('father', 'mother'), ('father', 'mother'), ('father', 'mother'), ('brother', 'sister'), ('brother', 'sister'), ('boy', 'girl'), ('man', 'woman'), ('man', 'wom

In [13]:
sameLevel = {
    "siblings": {"brother", "sister"},
    "parent_child": {"father", "mother", "son", "daughter"},
    "spouses": {"husband", "wife"},
    "royalty": {"king", "queen", "prince", "princess"},
    "extended_family": {"uncle", "aunt", "nephew", "niece"},
    "personal_pronouns": {"he", "she", "him", "her", "his", "hers"},
    "titles": {"gentleman", "lady", "sir", "ma'am", "mr", "mrs", "madam"},
    "heroic_roles": {"hero", "heroine"},
    "social_roles": {"patriarch", "matriarch"},
    "general_plural": {"men", "women"},
}
flat = {word for group in sameLevel.values() for word in group}
def isSame(pair):
    return pair[0] in flat and pair[1] in flat
femaleFirst = [pair for pair in femaleFirst if isSame(pair)]

In [14]:
maleFirst = [pair for pair in maleFirst if isSame(pair)]
print(len(maleFirst))
print(len(femaleFirst))

244
183


# Sentiment analysis

In [33]:
fm = ', '.join(map(str, [item for sublist in menPara for item in sublist]))
fw = ', '.join(map(str, [item for sublist in womenPara for item in sublist]))
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()
fscores = analyzer.polarity_scores(fw)
mscores = analyzer.polarity_scores(fm)
print(fscores,mscores)

{'neg': 0.063, 'neu': 0.8, 'pos': 0.137, 'compound': 1.0} {'neg': 0.061, 'neu': 0.795, 'pos': 0.143, 'compound': 1.0}


# TFIDF

In [16]:
from langdetect import detect
from langdetect import LangDetectException
import nltk
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer

In [17]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('vader_lexicon')
nltk.download('punkt_tab')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package stopwords to /home/tal012/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/tal012/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/tal012/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package punkt_tab to /home/tal012/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/tal012/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [18]:
#stemming the words
from nltk.stem import PorterStemmer
stemmer = PorterStemmer()
def stem_nested_list(nested_list):
    return [
        [
            ' '.join(stemmer.stem(word) for word in sentence.split())
            for sentence in sublist
        ]
        for sublist in nested_list
    ]
menPara = stem_nested_list(menPara)
womenPara = stem_nested_list(womenPara)
fm = ', '.join(map(str, [item for sublist in menPara for item in sublist]))
fw = ', '.join(map(str, [item for sublist in womenPara for item in sublist]))

In [19]:
tobeTFIDF = [" ".join(" ".join(row) for row in menPara)," ".join(" ".join(row) for row in womenPara)]
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(tobeTFIDF)
feature_names = vectorizer.get_feature_names_out()
dense_tfidf = tfidf_matrix.toarray()

In [20]:
#tfidf of female words
doc_index = 1
doc_tfidf_scores = dense_tfidf[doc_index]
word_score_pairs = list(zip(feature_names, doc_tfidf_scores))
sorted_word_scores = sorted(word_score_pairs, key=lambda x: x[1], reverse=True)
top_300_words_fs = sorted_word_scores[:300]
top_300_words_f = [word for word, score in sorted_word_scores[:300]]

In [21]:
#tfidf of male words
doc_index = 0  
doc_tfidf_scores = dense_tfidf[doc_index]
word_score_pairs_m = list(zip(feature_names, doc_tfidf_scores))
sorted_word_scores_m = sorted(word_score_pairs_m, key=lambda x: x[1], reverse=True)
top_300_words_ms = sorted_word_scores_m[:300]
top_300_words_m = [word for word, score in sorted_word_scores_m[:300]]

In [22]:
#remove common words and gendered keywords
common = list(set(top_300_words_f) & set(top_300_words_m))
top_300_words_f = [word for word in top_300_words_f if word not in common]
top_300_words_f = [word for word in top_300_words_f if word not in womenWordsNoSpace]
female_300 = " ".join(top_300_words_f)
top_300_words_m = [word for word in top_300_words_m if word not in common]
top_300_words_m = [word for word in top_300_words_m if word not in menWordsNoSpace]
male_300 = " ".join(top_300_words_m)

In [23]:
print(female_300)
print(male_300)

femal tv sport susan today verb beauti lesson parti ok earli usual emili often babi dad peter stevenson yesterday hope marriag photo st adj michael weekend bag pair tom dress niec enjoy lyn mum arriv bad french power problem care ed london nurs practic short someon train voic without busi danc done drink drive grammar near note sing sometim stand wash wonder birthday
princ romeo juliet arthur kill dead tree god ben wise water face lawrenc order becom shall april pari cloth england film sound carri heard soon found land leon open laugh ten hermit sword draft dream gave lo around cri goe head white hour later street might polic task idea bed bediver mark match met six summer true modr game martin news noth


In [24]:
from collections import Counter
import re
mwords = male_300.split()
filteredMwords = [word for word in mwords if word in fm]
wordMcounts = Counter(filteredMwords)
topMwords = wordMcounts.most_common(40)

In [25]:
fwords = female_300.split()
filteredFwords = [word for word in fwords if word in fw]
wordFcounts = Counter(filteredFwords)
topFwords = wordFcounts.most_common(40)

In [26]:
from collections import Counter
import re

# Example word lists (assuming male_300 and female_300 are defined as strings)
mwords = male_300.split()  # Male words
fwords = female_300.split()  # Female words

# Filtered words for male
filteredMwords = [word for word in mwords if word in fm]
wordMcounts = Counter(filteredMwords)
topMwords = wordMcounts.most_common(40)
print("Top 40 Male Words:", topMwords)

# Filtered words for female
filteredFwords = [word for word in fwords if word in fw]
wordFcounts = Counter(filteredFwords)
topFwords = wordFcounts.most_common(40)
print("Top 40 Female Words:", topFwords)


Top 40 Male Words: [('princ', 1), ('romeo', 1), ('juliet', 1), ('arthur', 1), ('kill', 1), ('dead', 1), ('tree', 1), ('god', 1), ('ben', 1), ('wise', 1), ('water', 1), ('face', 1), ('lawrenc', 1), ('order', 1), ('becom', 1), ('shall', 1), ('april', 1), ('pari', 1), ('cloth', 1), ('england', 1), ('film', 1), ('sound', 1), ('carri', 1), ('heard', 1), ('soon', 1), ('found', 1), ('land', 1), ('leon', 1), ('open', 1), ('laugh', 1), ('ten', 1), ('hermit', 1), ('sword', 1), ('draft', 1), ('dream', 1), ('gave', 1), ('lo', 1), ('around', 1), ('cri', 1), ('goe', 1)]
Top 40 Female Words: [('femal', 1), ('tv', 1), ('sport', 1), ('susan', 1), ('today', 1), ('verb', 1), ('beauti', 1), ('lesson', 1), ('parti', 1), ('ok', 1), ('earli', 1), ('usual', 1), ('emili', 1), ('often', 1), ('babi', 1), ('dad', 1), ('peter', 1), ('stevenson', 1), ('yesterday', 1), ('hope', 1), ('marriag', 1), ('photo', 1), ('st', 1), ('adj', 1), ('michael', 1), ('weekend', 1), ('bag', 1), ('pair', 1), ('tom', 1), ('dress', 1), 

# random forest

In [37]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
import numpy as np
import pandas as pd
male_segments = fm.split(" ")  
female_segments = fw.split(" ")  
texts = male_segments + female_segments  
labels = [0] * len(male_segments) + [1] * len(female_segments)  
vectorizer = TfidfVectorizer()  
X = vectorizer.fit_transform(texts)  
feature_names = vectorizer.get_feature_names_out()

clf = RandomForestClassifier(random_state=42, n_estimators=100)
clf.fit(X, labels)


proba = clf.predict_proba(X)[:, 1]  # Probability of being female
female_contributions = np.dot(X.T.toarray(), proba)
female_importance = female_contributions / np.sum(X.toarray(), axis=0)
female_importance_df = pd.DataFrame({
    "Word": feature_names,
    "FemaleImportance": female_importance
}).sort_values(by="FemaleImportance", ascending=False)

print("Top words most related to Female:")
print(female_importance_df.head(10))


Top words most related to Female:
            Word  FemaleImportance
1766        cass               1.0
9332   portugues               1.0
499         anya               1.0
686       aspect               1.0
11166     soccer               1.0
35      aborigin               1.0
1666      calmli               1.0
4358       fridg               1.0
13654     xavier               1.0
11675     sungen               1.0


In [40]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
import numpy as np
import pandas as pd
male_segments = fm.split(" ")  
female_segments = fw.split(" ")  
texts = male_segments + female_segments  
labels = [0] * len(male_segments) + [1] * len(female_segments)  
vectorizer = TfidfVectorizer()  
X = vectorizer.fit_transform(texts)  
feature_names = vectorizer.get_feature_names_out()

clf = RandomForestClassifier(random_state=42, n_estimators=100)
clf.fit(X, labels)

proba = clf.predict_proba(X)[:, 0]  # Probability of being male (class 0)


male_contributions = np.dot(X.T.toarray(), proba)
male_importance = male_contributions / np.sum(X.toarray(), axis=0)
male_importance_df = pd.DataFrame({
    "Word": feature_names,
    "MaleImportance": male_importance
}).sort_values(by="MaleImportance", ascending=False)

print("Top words most related to Male:")
print(male_importance_df.head(10))


Top words most related to Male:
           Word  MaleImportance
4070      fierc             1.0
5479     ibalon             1.0
5132     hermit             1.0
4803    grendel             1.0
425    ancestor             1.0
4620    gisborn             1.0
4593       giao             1.0
10659       sec             1.0
4565     gestur             1.0
10675      seed             1.0
